# Janta_Hack Computer Vision

#### Importing important Libraries:

In [1]:
import pandas as pd 
import numpy as np 
import os 

pd.options.display.max_columns = 50 
pd.get_option('display.max_rows') 
pd.set_option('display.max_rows',None) 

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop 

import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.callbacks import ReduceLROnPlateau


C:\Anaconda\envs\DeepLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda\envs\DeepLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda\envs\DeepLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda\envs\DeepLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:529: 

# Data Pre-Processing

In [3]:
# Path for Images.
image_path = 'images/'

In [4]:
# Get train and test data sets.
train = pd.read_csv('train.csv')
test = pd.read_csv('test_vc2kHdQ.csv')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646 entries, 0 to 1645
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   image_names       1646 non-null   object
 1   emergency_or_not  1646 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 25.8+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706 entries, 0 to 705
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_names  706 non-null    object
dtypes: object(1)
memory usage: 5.6+ KB


In [7]:
train.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [8]:
test.head()

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [9]:
train.describe(include='all')

,image_names,emergency_or_not
count,1646,1646.000000
unique,1646,NaN
top,569.jpg,NaN
freq,1,NaN
mean,NaN,0.413730
std,NaN,0.492651
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,1.000000


In [10]:
test.describe(include ='all')

,image_names
count,706
unique,706
top,211.jpg
freq,1


In [12]:
#Get the images from path using CSV names.
def get_images_list(path):
    image_list = []
    for img in tqdm(os.listdir(path)):
        image_list.append(img) 
    return image_list

In [13]:
# Use function to get the list 
image_list = get_images_list(image_path)

100%|██████████| 2352/2352 [00:00<00:00, 624288.26it/s]


In [17]:
image_list

['0.jpg',
 '1.jpg',
 '10.jpg',
 '100.jpg',
 '1000.jpg',
 '1001.jpg',
 '1002.jpg',
 '1003.jpg',
 '1004.jpg',
 '1005.jpg',
 '1006.jpg',
 '1007.jpg',
 '1008.jpg',
 '1009.jpg',
 '101.jpg',
 '1010.jpg',
 '1011.jpg',
 '1012.jpg',
 '1013.jpg',
 '1014.jpg',
 '1015.jpg',
 '1016.jpg',
 '1017.jpg',
 '1018.jpg',
 '1019.jpg',
 '102.jpg',
 '1020.jpg',
 '1021.jpg',
 '1022.jpg',
 '1023.jpg',
 '1024.jpg',
 '1025.jpg',
 '1026.jpg',
 '1027.jpg',
 '1028.jpg',
 '1029.jpg',
 '103.jpg',
 '1030.jpg',
 '1031.jpg',
 '1032.jpg',
 '1033.jpg',
 '1034.jpg',
 '1035.jpg',
 '1036.jpg',
 '1037.jpg',
 '1038.jpg',
 '1039.jpg',
 '104.jpg',
 '1040.jpg',
 '1041.jpg',
 '1042.jpg',
 '1043.jpg',
 '1044.jpg',
 '1045.jpg',
 '1046.jpg',
 '1047.jpg',
 '1048.jpg',
 '1049.jpg',
 '105.jpg',
 '1050.jpg',
 '1051.jpg',
 '1052.jpg',
 '1053.jpg',
 '1054.jpg',
 '1055.jpg',
 '1056.jpg',
 '1057.jpg',
 '1058.jpg',
 '1059.jpg',
 '106.jpg',
 '1060.jpg',
 '1061.jpg',
 '1062.jpg',
 '1063.jpg',
 '1064.jpg',
 '1065.jpg',
 '1066.jpg',
 '1067.jpg',
 

In [31]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


In [43]:
from keras.preprocessing import image
## Load training images from the given path based on image names which are there in train data and convert them into array.
def load_train_data(img_path):
    train_data = [] ## Initialize empty list
    for img in tqdm(train['image_names']): ## Get list of image names from train data and process each image name.
        if img in image_list: ## If the image name is present in image list then only we have to read image.
            path = os.path.join(img_path, img) ## Location of the the image.
            img = image.load_img(path,                              ## Load image from the given path and
                                 target_size=(64,64,3), ## Keep image size as 28X28X3(height,width,color channels) and
                                 grayscale=False)                   ## grayscale is false indicates that image is color image.
            img = image.img_to_array(img) ## Convert image pixels into an array.
            img = img/255 ## Normalize the train data (CNN converg faster on [0..1] data than on [0..255]).
            train_data.append(img) ## Add normalized image pixel array to train data.      
    return np.array(train_data) ## Convert list into an array and returns it.

In [44]:
## Get training data for the given image path.
train_data = load_train_data(image_path)

100%|██████████| 1646/1646 [00:01<00:00, 943.80it/s]


In [45]:
## Load testing images from the given path based on images names which are there in test data and convert them into array.
def load_test_data(image_path):
    test_data = []  ## Initialize empty list
    for img in tqdm(test['image_names']): ## Get list of image names from test data and process each image name.
        if img in image_list: ## If the image name is present in image list then only we have to read image.
            path = os.path.join(image_path, img) ## Location of the the image.
            img = image.load_img(path,                              ## Load image from the given path and
                                 target_size=(64,64,3), ## Keep image size as 28X28X3(height,width,color channels) and
                                 grayscale=False)                   ## grayscale is false indicates that image is color image.
            img = image.img_to_array(img) ## Convert image pixels into an array.
            img = img/255 ## Normalize the test data (CNN converg faster on [0..1] data than on [0..255]).
            test_data.append(img) ## Add normalized image pixel array to test data.
    return np.array(test_data) ## Convert list into an array and returns it.

In [46]:
# Get testing data for the given image path.
test_data = load_test_data(image_path)

100%|██████████| 706/706 [00:00<00:00, 940.18it/s]


In [136]:
from keras.utils import to_categorical ## One hot Encoding.
# Do one hot encoding on target/label varible.
y = train['emergency_or_not'].values ## Fetch label/target values(0/1).
y = to_categorical(y) ## Converts a class vector (integers) to binary class matrix.

## Split the train data into train and validation data.

In [51]:

X_train, X_test, y_train, y_test = train_test_split(train_data,       ## Features(i/p).
                                                    y,                ## Traget(0/p).
                                                    random_state=101, ## It is the seed used by the random number generator.
                                                    test_size=0.2)    ## % of train and validation division.(80:20)

In [57]:
X_train[0]

array([[[0.4745098 , 0.49019608, 0.5254902 ],
        [0.4392157 , 0.45490196, 0.49019608],
        [0.42352942, 0.4392157 , 0.4745098 ],
        ...,
        [0.68235296, 0.90588236, 0.98039216],
        [0.6745098 , 0.9019608 , 0.9882353 ],
        [0.6901961 , 0.8862745 , 1.        ]],

       [[0.49803922, 0.5254902 , 0.5647059 ],
        [0.5176471 , 0.54509807, 0.58431375],
        [0.43529412, 0.4627451 , 0.5019608 ],
        ...,
        [0.69803923, 0.9137255 , 0.99215686],
        [0.70980394, 0.9098039 , 0.99215686],
        [0.7176471 , 0.8901961 , 0.9882353 ]],

       [[0.48235294, 0.5137255 , 0.5568628 ],
        [0.5647059 , 0.59607846, 0.6392157 ],
        [0.48235294, 0.5137255 , 0.5568628 ],
        ...,
        [0.76862746, 0.92941177, 1.        ],
        [0.7529412 , 0.8980392 , 0.972549  ],
        [0.78431374, 0.89411765, 0.98039216]],

       ...,

       [[0.12941177, 0.15686275, 0.19607843],
        [0.09803922, 0.1254902 , 0.16470589],
        [0.1882353 , 0

In [58]:
y_train[0]

array([1., 0.], dtype=float32)

# Approach : 1 (With Out Data Augmentation)

In [59]:
## Part 2 - Building the CNN

### Initialising the CNN

cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

cnn.add(tf.keras.layers.Conv2D(filters=32, 
                               kernel_size=3, 
                               activation='relu', 
                               input_shape=[64, 64, 3]))

### Step 2 - Pooling

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Droput 25% Nodes.
cnn.add(tf.keras.layers.Dropout(0.25))


### Step 3 - Flattening

cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

cnn.add(tf.keras.layers.Dense(units=2, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [60]:
### Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Train on 1316 samples, validate on 330 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
1316/1316 [==============================] - 5s 3ms/sample - loss: 0.6004 - acc: 0.6801 - val_loss: 0.4773 - val_acc: 0.7348
Epoch 2/30
1316/1316 [==============================] - 4s 3ms/sample - loss: 0.4616 - acc: 0.7964 - val_loss: 0.4756 - val_acc: 0.7561
Epoch 3/30
1316/1316 [==============================] - 4s 3ms/sample - loss: 0.4152 - acc: 0.8123 - val_loss: 0.4529 - val_acc: 0.7773
Epoch 4/30
1316/1316 [==============================] - 4s 3ms/sample - loss: 0.3792 - acc: 0.8359 - val_loss: 0.4327 - val_acc: 0.7833
Epoch 5/30
1316/1316 [==============================] - 4s 3ms/sample - loss: 0.3194 - acc: 0.8685 - val_loss: 0.4596 - val_acc: 0.7833
Epoch 6/30
1316/1316 [==============================] - 4s 3ms/sample - loss: 0.3124 - acc: 0.8758 - val_loss: 0.4369 - val_acc: 0.7848
Epoch 7/30
1316/1316 [==============================] - 4s 3ms/sample - loss: 0.2576 - ac

In [61]:
## Get predictions for test data
pred_cnn = cnn.predict_classes(test_data)

In [63]:
sub_cnn = pd.read_csv('sample_submission_yxjOnvz.csv')

In [65]:
sub_cnn['emergency_or_not'] = pred_cnn

In [66]:
sub_cnn.to_csv('cnn_try.csv',index = False)

In [41]:
# Compile the model.
model.compile(loss='categorical_crossentropy',  ## String (name of objective function) or objective function or`Loss` instance.
              optimizer='Adam',                 ## String (name of optimizer) or optimizer instance.
              metrics=['accuracy'])             ## List of metrics to be evaluated by the model during training and testing.

# Approach : 2 (With Data Augmentation)

In [74]:
## Part 2 - Building the CNN

### Initialising the CNN

cnn1 = tf.keras.models.Sequential()

### Step 1 - Convolution

cnn1.add(tf.keras.layers.Conv2D(filters=32, 
                               kernel_size=5,
                               padding = 'same',
                               activation='relu', 
                               input_shape=[64, 64, 3]))

### Step 2 - Pooling

cnn1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

cnn1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=5, activation='relu'))
cnn1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Droput 25% Nodes.
cnn1.add(tf.keras.layers.Dropout(0.25))

cnn1.add(tf.keras.layers.Conv2D(filters=32, 
                               kernel_size=3,
                               padding = 'same',
                               activation='relu', 
                               input_shape=[64, 64, 3]))

cnn1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Droput 25% Nodes.
cnn1.add(tf.keras.layers.Dropout(0.25))


### Step 3 - Flattening

cnn1.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

cnn1.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

cnn1.add(tf.keras.layers.Dense(units=2, activation='sigmoid'))



In [83]:
## Part 3 - Training the CNN

### Compiling the CNN

cnn1.compile(optimizer ='adam',           
              loss = "categorical_crossentropy", 
              metrics=["accuracy"])


In [84]:
datagen = ImageDataGenerator(featurewise_center=False,            ## Set input mean to 0 over the dataset
                             samplewise_center=False,             ## Set each sample mean to 0
                             featurewise_std_normalization=False, ## Divide inputs by std of the dataset
                             samplewise_std_normalization=False,  ## Divide each input by its std
                             zca_whitening=False,                 ## Apply ZCA whitening
                             rotation_range=10,                   ## Randomly rotate images in the range (degrees, 0 to 180)
                             zoom_range = 0.1,                    ## Randomly zoom image 
                             width_shift_range=0.1,               ## Randomly shift images horizontally (fraction of total width)
                             height_shift_range=0.1,              ## Randomly shift images vertically (fraction of total height)
                             horizontal_flip=False,               ## Randomly flip images horizontally
                             vertical_flip=False)                 ## Randomly flip images vertically

## Fit data augmentation model
datagen.fit(X_train)

In [88]:
## Fit the model
history = cnn1.fit_generator(datagen.flow(X_train,                           ## Input data
                                           y_train,                           ## Labels/ Target/ out put data
                                           batch_size=32),            ## Batch size (default: 32)
                              epochs = 30,                                ## Number of epochs to train the model.
                              validation_data = (X_test, y_test),             ## On which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data(Validation data).
                              verbose = 2,                                    ## 0, 1, or 2. Verbosity mode 0 = silent, 1 = progress bar, 2 = one line per epoch.
                              steps_per_epoch=X_train.shape[0] // batch_size)  

Epoch 1/30
330/330 [==============================] - 1s 2ms/sample - loss: 0.6441 - acc: 0.6879
 - 7s - loss: 0.6855 - acc: 0.5600 - val_loss: 0.6423 - val_acc: 0.6879
Epoch 2/30
330/330 [==============================] - 1s 2ms/sample - loss: 0.5976 - acc: 0.7152
 - 7s - loss: 0.6435 - acc: 0.6505 - val_loss: 0.5995 - val_acc: 0.7152
Epoch 3/30
330/330 [==============================] - 1s 2ms/sample - loss: 0.5721 - acc: 0.7273
 - 7s - loss: 0.6514 - acc: 0.6383 - val_loss: 0.5744 - val_acc: 0.7273
Epoch 4/30
330/330 [==============================] - 1s 2ms/sample - loss: 0.5520 - acc: 0.7424
 - 7s - loss: 0.5728 - acc: 0.7105 - val_loss: 0.5505 - val_acc: 0.7424
Epoch 5/30
330/330 [==============================] - 1s 2ms/sample - loss: 0.5665 - acc: 0.7212
 - 7s - loss: 0.5588 - acc: 0.7371 - val_loss: 0.5732 - val_acc: 0.7212
Epoch 6/30
330/330 [==============================] - 1s 2ms/sample - loss: 0.5150 - acc: 0.7394
 - 7s - loss: 0.5368 - acc: 0.7424 - val_loss: 0.5179 - va

In [113]:
cnn1_pred = cnn1.predict(test_data)

In [132]:
cnn1_pred = np.argmax(cnn1_pred,axis = 1)

array([0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,

In [133]:
sub_cnn1 = pd.read_csv('sample_submission_yxjOnvz.csv')
sub_cnn1['emergency_or_not'] = cnn1_pred
sub_cnn1.to_csv('cnn_pred_1.csv',index = False)